In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
#Read training data
df_train = pd.read_csv('train_hire_stats_719.csv')
df_train.shape

In [0]:
#Check weekday & workday
#holidays連假的平常日
#workdays連假的補班日
#malfunction
holidays= {'2016-02-08', '2016-02-09', '2016-02-10', '2016-02-11', '2016-02-12','2016-02-29', '2016-04-04', '2016-04-05', '2016-06-09', '2016-06-10', 
'2016-09-15', '2016-09-16', '2016-10-10', '2017-01-02', '2017-01-27', '2017-01-30', '2017-01-31', '2017-02-01','2017-02-27', '2017-02-28'}
makeupworkdays = {'2016-06-04', '2016-09-10', '2017-02-18'}
malfunctiondays = {'2016-02-05', '2016-02-16', '2016-03-06', '2016-05-2', '2016-07-08', '2016-08-12', '2016-08-15', '2016-09-27', '2016-09-28'}
#malfunctiondays = {'2016-02-05', '2016-02-16'}

In [0]:
len(df_train)

In [0]:
df_train.head()

In [0]:
#Declare weekday & workday
isworkday = np.ones((len(df_train),), dtype=int)
weekday = np.ones((len(df_train),), dtype=int)
malday = np.zeros((len(df_train),), dtype=int)

In [0]:
#Compute weekday & workday
#weekday()
#0 == Monday, #1 == Tuesday, #2 == Wednesday,  #3 == Thursday,  #4 == Friday,  #5 == Saturday, #6 == Sunday
from datetime import datetime
from datetime import timedelta

for index, row in df_train.iterrows():
    dd=datetime.strptime(row['Date'], "%Y/%m/%d")
    hh = int(row['Hour_slot'])
    if hh < 8:
      dd = dd - timedelta(days=1)
    thedaystr =  dd.strftime("%Y-%m-%d")
    if thedaystr in holidays:
        isworkday[index] = 0
    else:
        weekday[index]= dd.weekday() 
        if weekday[index] >=5 and thedaystr not in makeupworkdays:
            isworkday[index] = 0
    if thedaystr in malfunctiondays:
      malday[index] = 1

In [0]:
#Build a new dataframe from the training data
RawX = pd.DataFrame(df_train[["Zone_ID", "Date", "Hour_slot"]])
RawX['isworkday'] = isworkday
RawX['weekday'] = weekday
RawX['malday'] = malday
RawX.shape

In [0]:
#Resize y into (0,1)
y0 = df_train["Hire_count"].values
y = df_train["Hire_count"].values/339.0
y.shape

In [0]:
plt.plot(y0)

In [0]:
y0= y0[RawX['malday']==0]
y = y[RawX['malday']==0]
y.shape

In [0]:
RawX = RawX[RawX['malday']==0]
RawX.shape

In [0]:
RawX=RawX.drop(columns=['malday'])
RawX.shape

In [0]:
RawOrg = pd.DataFrame(RawX)
RawOrg.shape

In [0]:
#See the raw input data
RawX.head()

In [0]:
RawX2 = pd.DataFrame(RawX)

In [0]:
RawX=RawX.drop(columns=['Date'])

In [0]:
#Use OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

In [0]:
#Build encoder
enc.fit_transform(RawX)
enc.categories_

In [0]:
#Transform data into one hot vector
X = enc.transform(RawX).toarray()
X.shape

In [0]:
#See the cooked input data
X[0:3, :]

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout

In [0]:
# create and fit a Baseline Neural Network Model
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=42))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1))

In [0]:
#myoptimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)
myoptimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0001)
model.compile(loss='mse', optimizer=myoptimizer)
model.fit(X, y, epochs=100, batch_size=1024, verbose=1)

In [0]:
yhat = model.predict(X)*339.0

In [0]:
plt.plot(yhat)

In [0]:
CookX = pd.DataFrame(RawOrg)
CookX['y0'] = y0
CookX['yhat'] = yhat
CookX = CookX[abs(CookX['y0']-CookX['yhat']) <50]
CookX.shape

In [0]:
CookX.shape

In [0]:
RawX2 = pd.DataFrame(CookX)
RawX=CookX.drop(columns=['y0','yhat','Date'])
RawX.shape

In [0]:
X2 = enc.transform(RawX).toarray()
X2.shape

In [0]:
y2 = RawX2["y0"].values/339.0
y2.shape

In [0]:
plt.plot(y)

In [0]:
# create and fit a Baseline Neural Network Model
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=42))
model.add(Dropout(0.3))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))

In [0]:
#myoptimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)
myoptimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0001)
model.compile(loss='mse', optimizer=myoptimizer)
model.fit(X2, y2, epochs=100, batch_size=1024, verbose=1)

In [0]:
yhat = model.predict(X2)*339.0

In [0]:
for i in range(len(yhat)):
  if yhat[i] < 0.0:
    yhat[i] = 0.0

In [0]:
dfhat = pd.DataFrame(yhat)
dfhat.to_csv('prediction5.csv')

In [0]:
RawX2["count"]=y2*339.0
RawX2["prediction"]=yhat

In [0]:
RawX2.to_csv('TrainModel1113B.csv')

In [0]:
model.save('model_htchu_20191113D.h5')

In [0]:
df_test = pd.read_csv('test_hire_stats.csv')
df_test.shape

In [0]:
#Declare weekday & workday
isworkday2 = np.ones((len(df_test),), dtype=int)
weekday2 = np.ones((len(df_test),), dtype=int)

In [0]:
for index, row in df_test.iterrows():
    dd=datetime.strptime(row['Date'], "%Y/%m/%d")
    hh = int(row['Hour_slot'])
    if hh < 8:
      dd = dd - timedelta(days=1)
    thedaystr =  dd.strftime("%Y-%m-%d")
    if thedaystr in holidays:
        isworkday2[index] = 0
    else:
        weekday2[index]= dd.weekday() 
        if weekday2[index] >=5 and thedaystr not in makeupworkdays:
            isworkday2[index] = 0

In [0]:
Test = pd.DataFrame(df_test[["Zone_ID", "Hour_slot"]])
Test['isworkday'] = isworkday2
Test['weekday'] = weekday2

In [0]:
Xtest = enc.transform(Test).toarray()
Xtest.shape

In [0]:
yt = model.predict(Xtest)*339.0

In [0]:
for i in range(len(yt)):
  if yt[i] < 0.0:
    yt[i] = 0.0

In [0]:
plt.plot(yt)

In [0]:
dfOut = pd.DataFrame(yt)

In [0]:
dfOut.to_csv('Result7.csv')